# scraping
https://non-dimension.com/kabuka-scraping/#toc2 より  
最後のセル”まとめコード”でほしい銘柄のすべての年度に渡ってスクレイピングを実行できます．

In [9]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import datetime

## ページのhtmlを取得
取得したい銘柄の株価データは、  
https://kabuoji3.com/stock/以下に”/銘柄コード/年”  
という形式で存在しているようです。

In [3]:
y = 2019 #2019年
stock_number = 6577 #好きな銘柄コード、今回はベスト・ワンドットコム
url = 'https://kabuoji3.com/stock/{}/{}/'.format(stock_number,y)
headers = {
      "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15 "
}
soup = BeautifulSoup(requests.get(url, headers = headers).content,'html.parser')

上記のようにBeautifulSoupでページのHTMLソースを取得すると以下のようなHTMLが得られます。

In [5]:
print(soup)


<td>3275</td>
<td>3300</td>
<td>8800</td>
<td>3300</td>
</tr>

<tr>
<td>2019-07-17</td>
<td>3250</td>
<td>3295</td>
<td>2990</td>
<td>3260</td>
<td>36600</td>
<td>3260</td>
</tr>

<tr>
<td>2019-07-18</td>
<td>3210</td>
<td>3210</td>
<td>3140</td>
<td>3150</td>
<td>4500</td>
<td>3150</td>
</tr>

<tr>
<td>2019-07-19</td>
<td>3150</td>
<td>3420</td>
<td>3150</td>
<td>3360</td>
<td>19300</td>
<td>3360</td>
</tr>

<tr>
<td>2019-07-22</td>
<td>3290</td>
<td>3425</td>
<td>3275</td>
<td>3400</td>
<td>7500</td>
<td>3400</td>
</tr>

<tr>
<td>2019-07-23</td>
<td>3395</td>
<td>3565</td>
<td>3380</td>
<td>3525</td>
<td>16600</td>
<td>3525</td>
</tr>

<tr>
<td>2019-07-24</td>
<td>3575</td>
<td>3690</td>
<td>3550</td>
<td>3550</td>
<td>26900</td>
<td>3550</td>
</tr>

<tr>
<td>2019-07-25</td>
<td>3570</td>
<td>3635</td>
<td>3495</td>
<td>3580</td>
<td>5200</td>
<td>3580</td>
</tr>

<tr>
<td>2019-07-26</td>
<td>3560</td>
<td>3590</td>
<td>3530</td>
<td>3570</td>
<td>4200</td>
<td>3570</td>
</tr>

<tr>

## 株価データをデータフレーム化
ページのHTMLの構成を頑張って調べていきます。
調べると割と整ったテーブルになっており、trタグの中のthタグにテーブルのヘッドがあり、trタグの中のtdタグにデータが入っている構成でした。

In [6]:
tag_tr = soup.find_all('tr')
head = [h.text for h in tag_tr[0].find_all('th')] #テーブルのヘッドの取得

#テーブルの各データの取得
data = []
for i in range(1,len(tag_tr)):
    data.append([d.text for d in tag_tr[i].find_all('td')])
    df = pd.DataFrame(data, columns = head)

## テキストデータを数値、タイムスタンプに変換

上で作成したdfを表示すると以下のようになっており、無事データフレーム化できたように思うのですが、見た目にはわからないのですが、実はすべてテキストデータなので、このままだとグラフを書いたりする時に扱いづらいままです。

In [7]:
print(df)

             日付    始値    高値    安値    終値    出来高    終値調整
0    2019-01-04  4210  4440  4105  4395   3400  2197.5
1    2019-01-07  4420  4780  4420  4780   6200    2390
2    2019-01-08  4920  5480  4920  5480   9800    2740
3    2019-01-09  5480  6440  5320  5700  30100    2850
4    2019-01-10  5600  5800  5550  5550   7000    2775
..          ...   ...   ...   ...   ...    ...     ...
236  2019-12-24  4370  5070  4200  5070  82200    5070
237  2019-12-25  4800  5090  4760  5040  59400    5040
238  2019-12-26  5010  5060  4865  5010  24400    5010
239  2019-12-27  4985  5410  4960  5120  30000    5120
240  2019-12-30  4985  5030  4710  5000  33000    5000

[241 rows x 7 columns]


したがって、数字をfloatに、日付をタイムスタンプに変換します。

In [8]:
col = ['始値','高値','安値','終値','出来高','終値調整']
for c in col:
    df[c] = df[c].astype(float)
df['日付'] = [datetime.strptime(i,'%Y-%m-%d') for i in df['日付']]

dfを再度表示すると以下のように数字の後ろに小数点が現れていると思います。（今回はわかりやすいようにfloatにしましたが、実際はintでも問題ありません）
見た目ではわかりませんが、日付もタイムスタンプに変換されているはずです。

In [9]:
print(df)

            日付      始値      高値      安値      終値      出来高    終値調整
0   2019-01-04  4210.0  4440.0  4105.0  4395.0   3400.0  2197.5
1   2019-01-07  4420.0  4780.0  4420.0  4780.0   6200.0  2390.0
2   2019-01-08  4920.0  5480.0  4920.0  5480.0   9800.0  2740.0
3   2019-01-09  5480.0  6440.0  5320.0  5700.0  30100.0  2850.0
4   2019-01-10  5600.0  5800.0  5550.0  5550.0   7000.0  2775.0
..         ...     ...     ...     ...     ...      ...     ...
236 2019-12-24  4370.0  5070.0  4200.0  5070.0  82200.0  5070.0
237 2019-12-25  4800.0  5090.0  4760.0  5040.0  59400.0  5040.0
238 2019-12-26  5010.0  5060.0  4865.0  5010.0  24400.0  5010.0
239 2019-12-27  4985.0  5410.0  4960.0  5120.0  30000.0  5120.0
240 2019-12-30  4985.0  5030.0  4710.0  5000.0  33000.0  5000.0

[241 rows x 7 columns]


## 複数年のデータの取得

このサイトでは株価データが年ごとに別ページにわかれているので、ページをfor文で繰り返して取得する必要があります。あと、最終的にはすべての年を1つのデータフレームにしたいので、結合したほうが良いと思います。ということで以下のようなコードになります。

In [10]:
#複数年のデータフレームの作成
dfs = []
year = [2018,2019]
for y in year:
    url = 'https://kabuoji3.com/stock/{}/{}/'.format(stock_number,y)
    headers = {
      "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15 "
      }
    soup = BeautifulSoup(requests.get(url, headers = headers).content,'html.parser')
    tag_tr = soup.find_all('tr')
    head = [h.text for h in tag_tr[0].find_all('th')]
    data = []
    for i in range(1,len(tag_tr)):
        data.append([d.text for d in tag_tr[i].find_all('td')])
    df = pd.DataFrame(data, columns = head)

    col = ['始値','高値','安値','終値','出来高','終値調整']
    for c in col:
        df[c] = df[c].astype(float)
    df['日付'] = [datetime.strptime(i,'%Y-%m-%d') for i in df['日付']]
    dfs.append(df)
 
#複数年のデータフレームの結合
data = pd.concat(dfs,axis=0)
data = data.reset_index(drop=True)

In [11]:
print(df)

            日付      始値      高値      安値      終値      出来高    終値調整
0   2019-01-04  4210.0  4440.0  4105.0  4395.0   3400.0  2197.5
1   2019-01-07  4420.0  4780.0  4420.0  4780.0   6200.0  2390.0
2   2019-01-08  4920.0  5480.0  4920.0  5480.0   9800.0  2740.0
3   2019-01-09  5480.0  6440.0  5320.0  5700.0  30100.0  2850.0
4   2019-01-10  5600.0  5800.0  5550.0  5550.0   7000.0  2775.0
..         ...     ...     ...     ...     ...      ...     ...
236 2019-12-24  4370.0  5070.0  4200.0  5070.0  82200.0  5070.0
237 2019-12-25  4800.0  5090.0  4760.0  5040.0  59400.0  5040.0
238 2019-12-26  5010.0  5060.0  4865.0  5010.0  24400.0  5010.0
239 2019-12-27  4985.0  5410.0  4960.0  5120.0  30000.0  5120.0
240 2019-12-30  4985.0  5030.0  4710.0  5000.0  33000.0  5000.0

[241 rows x 7 columns]


## 取得した株価の銘柄リストを作成する

自分が取得したい株価の銘柄リストを作成します。
作成したリストをcsvにして以下のコードで読み込みます。

In [10]:
code_list = pd.read_csv('code_list.csv')
code_list.head(10)

,code,name,連動対象,取引値,売買単位,運用会社,信託報酬（税抜）
0,1319,(NEXT FUNDS)日経300株価指数連動型上場投信,日経300,373,1000,野村,0.51%
1,1357,(NEXT FUNDS) 日経ダブルインバース上場投信,日経平均ダブルインバース,407,1,野村,0.80%
2,1311,(NEXT FUNDS)TOPIX Core30連動型上場投信,TOPIX Core30,913,10,野村,0.19%
3,1472,(NEXT FUNDS)JPX日経400ダブルインバース,JPX日経400ダブルインバース,922,1,野村,0.80%
4,2510,(NEXT FUNDS)国内債券・NOMURA-BPI総合,NOMURA-BPI総合,992,10,野村,0.07%
5,1571,(NEXT FUNDS)日経平均インバース上場投信,日経平均インバース,995,1,野村,0.80%
6,2529,(NEXT FUNDS)野村株主還元70連動型上場投信,野村株主還元70,"1,043",1,野村,0.28%
7,2518,(NEXT FUNDS) MSCI日本株女性活躍(セレクト)上場投信,MSCI日本株女性活躍セレクト,"1,120",1,野村,0.15%
8,1343,(NEXT FUNDS)東証REIT指数連動型上場投信,東証REIT指数,"2,071",10,野村,0.16%
9,1471,(NEXT FUNDS)JPX日経400インバース,JPX日経400インバース,"3,460",1,野村,0.80%


## まとめコード
最後に上記のコードを関数化し、複数の銘柄コードを取得するコードにしておきます。
古い年月まで含めるとデータがないですよと怒られることがあるので、try文を使ってエラー回避しています。最後にcsvで保存しておきます。

In [17]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import datetime

def get_dfs(stock_number):
    dfs = []
    #year = [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021] #2017〜2019年までの株価データを取得
    #for y in year:
    year_begin = 2015
    year_end = 2022
    for y in range(year_begin,year_end):
        try:
            print(y)
            url = 'https://kabuoji3.com/stock/{}/{}/'.format(stock_number,y)
            headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15 "
            }
            soup = BeautifulSoup(requests.get(url, headers = headers).content,'html.parser')
            tag_tr = soup.find_all('tr')
            head = [h.text for h in tag_tr[0].find_all('th')]
            data = []
            for i in range(1,len(tag_tr)):
                data.append([d.text for d in tag_tr[i].find_all('td')])
            df = pd.DataFrame(data, columns = head)

            col = ['始値','高値','安値','終値','出来高','終値調整']
            for c in col:
                df[c] = df[c].astype(float)
            df['日付'] = [datetime.strptime(i,'%Y-%m-%d') for i in df['日付']]
            dfs.append(df)
        except IndexError:
            print('No data')
    return dfs

def concatenate(dfs):
    data = pd.concat(dfs,axis=0)
    data = data.reset_index(drop=True)
    col = ['始値','高値','安値','終値','出来高','終値調整']
    for c in col:
        data[c] = data[c].astype(float)
    return data

#作成したコードリストを読み込む
code_list = pd.read_csv('code_list.csv')

#複数のデータフレームをcsvで保存
for i in range(len(code_list)):
    k = code_list.loc[i,'code']
    v = code_list.loc[i,'name']
    print(k,v)
    dfs = get_dfs(k)
    data = concatenate(dfs) 
    #data.to_csv('{}-{}.csv'.format(k,v))
    data.to_csv('{}.csv'.format(k))     #一行上の行でエラーはいたらこっち

1477 iシェアーズ MSCI 日本株最小分散 ETF
2015
2016
2017
2018
2019
2020
2021
1478 iシェアーズ MSCI ジャパン高配当利回り ETF
2015
2016
2017
2018
2019
2020
2021
1475 iシェアーズ TOPIX ETF
2015
2016
2017
2018
2019
2020
2021
